### Import Libraries

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix, recall_score, f1_score


import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import (
    InputLayer,
    Conv2D,
    MaxPooling2D,
    GlobalAveragePooling2D,
    Activation,
    Flatten,
    Dense,
    Dropout
)
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras.applications import VGG19, EfficientNetB0
from tensorflow.keras.optimizers import Adam

import os
from PIL import Image

is_gpu = tf.config.list_physical_devices('GPU')[0]
if is_gpu:
    print("GPU available")

import warnings
warnings.filterwarnings('ignore')

### Download Dataset if running on Google Colab

In [ ]:
if 'google.colab' in str(get_ipython()):
    print('Downloading cards image classification...\n')
    !mkdir -p data
    !curl -L -o data/cards-image-dataset.zip https://www.kaggle.com/api/v1/datasets/download/gpiosenka/cards-image-datasetclassification
    !unzip -q data/cards-image-dataset.zip -d data
    !rm data/cards-image-dataset.zip
    print('Dataset downloaded and extracted to data directory.')
    print('Download demo image for testing...')
    !mkdir -p uploads
    !curl -L -o "uploads/demo.jpg" https://thumbs.dreamstime.com/z/ten-clubs-card-clipping-path-one-series-images-showing-each-playing-standard-deck-all-have-easy-manipulation-270099775.jpg
    print('Demo image downloaded to uploads directory.')
else:
    print('Not running on Colab')

### Load Dataset

In [ ]:
data = pd.read_csv('data/cards.csv', delimiter=',')
data

### Data Preprocessing & Exploration

In [ ]:
print(f"Number of card types: {len(data['card type'].unique())}")
print(f"Number of card types with their suits: {len(data['labels'].unique())}")

In [ ]:
data['card type'] = data['card type'].replace('xxx', 'joker')

# show unique card types and their counts
card_types = data['card type'].value_counts()
print("Card Types and their counts:")
print(card_types)

# plot card types distribution
plt.figure(figsize=(12, 6))
sns.countplot(data=data, x='card type', order=card_types.index)
plt.title('Distribution of Card Types')
plt.xlabel('Card Type')
plt.ylabel('Count')
plt.xticks(rotation=45)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()


### Custom Models Functions

In [ ]:
def create_custom_model_v1(shape, num_classes=14) -> tf.keras.Model:
    custom_model = Sequential(
        [
            InputLayer(shape),
            Conv2D(16, (3, 3), padding='same', activation='relu'),
            MaxPooling2D((2, 2)),
            Flatten(),
            Dense(num_classes, activation="softmax"),
        ],
        name='custom_model_v1'
    )

    custom_model.summary()

    return custom_model


def create_custom_model_v2(shape, num_classes=14) -> tf.keras.Model:
    custom_model = Sequential(
        [
            InputLayer(shape),
            Conv2D(32, (3, 3), padding='same', activation='relu'),
            MaxPooling2D((2, 2)),

            Conv2D(64, (3, 3), padding='same', activation='relu'),
            MaxPooling2D((2, 2)),

            Conv2D(128, (3, 3), padding='same', activation='relu'),
            MaxPooling2D((2, 2)),

            Flatten(),
            Dense(num_classes, activation="softmax"),
        ],
        name='custom_model_v2')

    custom_model.summary()

    return custom_model


def create_custom_model_v3(shape, num_classes=14) -> tf.keras.Model:
    custom_model = Sequential(
        [
            InputLayer(shape),
            Conv2D(32, (3, 3)),

            Activation('relu'),
            MaxPooling2D((2, 2)),

            Conv2D(64, (3, 3)),

            Activation('relu'),
            MaxPooling2D((2, 2)),

            Conv2D(128, (3, 3)),

            Activation('relu'),
            MaxPooling2D((2, 2)),

            Conv2D(256, (3, 3)),

            Activation('relu'),
            MaxPooling2D((2, 2)),

            Flatten(),
            Dense(512, activation="relu"),
            Dropout(0.5),
            Dense(num_classes, activation="softmax"),
        ],
        name='custom_model_v3'
    )

    custom_model.summary()

    return custom_model


def create_pretrained_model_vgg19(shape=(224, 224, 3), num_classes=14) -> tf.keras.Model:
    base_model = VGG19(weights='imagenet', include_top=False, input_shape=shape)
    base_model.trainable = True

    for layer in base_model.layers[:-8]:  # keep early  layers frozen
        layer.trainable = False

    vgg = Sequential(
        [
            base_model,
            GlobalAveragePooling2D(),
            Dense(512, activation='relu'),
            Dropout(0.2),
            Dense(256, activation='relu'),
            Dropout(0.2),
            Dense(num_classes, activation='softmax')
        ],
        name='vgg19_pretrained_model'
    )

    vgg.summary()

    return vgg


def create_pretrained_model_resnet50(shape=(224, 224, 3), num_classes=14) -> tf.keras.Model:
    base_model = tf.keras.applications.ResNet50(weights='imagenet', include_top=False, input_shape=shape)
    base_model.trainable = True
    for layer in base_model.layers[:-15]:  # keep early layers frozen
        layer.trainable = False

    resnet50 = Sequential(
        [
            base_model,
            GlobalAveragePooling2D(),
            Dense(512, activation="relu"),
            Dropout(0.3),
            Dense(256, activation="relu"),
            Dropout(0.3),
            Dense(num_classes, activation="softmax"),
        ],
        name="resnet50_pretrained_model"
    )

    resnet50.summary()

    return resnet50

### Helper Functions

In [ ]:
def data_generator(data, batch_size=32, target_size=(224, 224), y_col='card type', use_augmentation=False):
    is_train_set = data['data set'].iloc[0] == 'train'
    if is_train_set and use_augmentation:
        # For training data, apply data augmentation if specified
        datagen = ImageDataGenerator(
            rescale=1. / 255, # Scales pixel values from the range [0, 255] to [0, 1]
            rotation_range=30, # Randomly rotates images by up to 30 degrees (either clockwise or counter-clockwise
            zoom_range=0.2, # Randomly zooms in/out inside images by up to 20%
            horizontal_flip=True, # Randomly flips images horizontally
            fill_mode='nearest', # Use the nearest pixel value to fill in the gap
        )
    else:
        # No augmentation for validation, test data and train data
        datagen = ImageDataGenerator(rescale=1. / 255)

    generator = datagen.flow_from_dataframe(
        dataframe=data,
        x_col='filepaths',
        y_col=y_col,
        directory='data',
        target_size=target_size,
        color_mode='rgb',
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=is_train_set,
        seed=42,
    )
    return generator


def plot_images(generator, num_images=8):
    images, labels = next(generator)
    plt.figure(figsize=(6, 6))
    for i in range(num_images):
        plt.subplot(2, 4, i + 1)
        plt.imshow(images[i])
        plt.title(list(generator.class_indices.keys())[np.argmax(labels[i])])
        plt.axis('off')
    plt.tight_layout()
    plt.show()


def get_optimizer(optimizer_name='adam', learning_rate=0.001) -> tf.keras.optimizers.Optimizer:
    params = {
        'class_name': optimizer_name,
        'config': {
            'learning_rate': learning_rate
        }
    }

    return tf.keras.optimizers.get(params)


def get_loss(loss_name='categorical_crossentropy') -> tf.keras.losses.Loss:
    loss = tf.keras.losses.get(loss_name)
    return loss


def prepare_model(model, optimizer='Adam', loss='categorical_crossentropy', learning_rate=0.001,
                  metrics=['accuracy']) -> tf.keras.Model:
    model.compile(
        optimizer=get_optimizer(optimizer, learning_rate),
        loss=get_loss(loss),
        metrics=metrics
    )
    return model


def early_stopping_callback(monitor='val_loss', patience=10) -> tf.keras.callbacks.EarlyStopping:
    return tf.keras.callbacks.EarlyStopping(
        monitor=monitor,
        patience=patience,
        restore_best_weights=True
    )


def model_checkpoint_callback(filepath='models/custom_model.h5',
                              monitor='val_loss') -> tf.keras.callbacks.ModelCheckpoint:
    return tf.keras.callbacks.ModelCheckpoint(
        filepath=filepath,
        monitor=monitor,
        save_best_only=True
    )


def train_model(model, train, validation, epochs=10, batch_size=32, use_early_stopping=None, save_name=None,
                verbose=1) -> tf.keras.callbacks.History:
    callbacks = []

    if use_early_stopping:
        callbacks.append(early_stopping_callback(monitor='val_accuracy', patience=5))

    if save_name:
        if not os.path.exists(f'models/labels_{model.output_shape[1]}'):
            os.makedirs(f'models/labels_{model.output_shape[1]}')

        callbacks.append(model_checkpoint_callback(filepath=f'models/labels_{model.output_shape[1]}/{save_name}.h5',
                                                   monitor='val_accuracy'))

    return model.fit(
        train,
        validation_data=validation,
        epochs=epochs,
        batch_size=batch_size,
        verbose=verbose,
        steps_per_epoch=len(train),
        validation_steps=len(validation),
        callbacks=callbacks
    )


def predict_model(model, generator, verbose=1) -> np.ndarray:
    return model.predict(generator, verbose=verbose)


def get_classification_report(model, generator) -> str:
    predictions = predict_model(model, generator)
    y_pred = np.argmax(predictions, axis=1)
    y_true = test_generator.classes
    print(
        classification_report(y_true, y_pred, target_names=list(test_generator.class_indices.keys()), zero_division=0))


def get_recall_score(model, generator):
    predictions = predict_model(model, generator, verbose=0)
    y_pred = np.argmax(predictions, axis=1)
    y_true = test_generator.classes
    return recall_score(y_true, y_pred, average='macro')


def get_f1_score(model, generator):
    predictions = predict_model(model, generator, verbose=0)
    y_pred = np.argmax(predictions, axis=1)
    y_true = test_generator.classes
    return f1_score(y_true, y_pred, average='macro')


def plot_confusion_matrix(model, generator, verbose=0) -> None:
    predictions = model.predict(generator, verbose=verbose)
    y_pred = np.argmax(predictions, axis=1)
    y_true = generator.classes
    cm = confusion_matrix(y_true, y_pred)
    class_names = list(generator.class_indices.keys())
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
    plt.title('Confusion Matrix')
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.show()


def plot_training_history(history):
    plt.figure(figsize=(12, 6))

    epochs = range(1, len(history.history['accuracy']) + 1)

    # Plot training accuracy
    plt.subplot(1, 2, 1)
    plt.plot(epochs, history.history['accuracy'], label='Train Accuracy')
    if 'val_accuracy' in history.history:
        plt.plot(epochs, history.history['val_accuracy'], label='Validation Accuracy')
        best_val_acc = max(history.history['val_accuracy'])
        best_epoch = history.history['val_accuracy'].index(best_val_acc) + 1  # +1 because epochs start at 1
        plt.plot(best_epoch, best_val_acc, 'ro')  # Highlight best validation accuracy
    plt.title('Training and Validation Accuracy')
    plt.xticks(epochs)
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()

    # Plot training loss
    plt.subplot(1, 2, 2)
    plt.plot(epochs, history.history['loss'], label='Train Loss')
    if 'val_loss' in history.history:
        plt.plot(epochs, history.history['val_loss'], label='Validation Loss')
    plt.title('Training and Validation Loss')
    plt.xticks(epochs)
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()

    plt.tight_layout()
    plt.show()

def print_model_performance(model, history, valid_generator, test_generator) -> None:
    best_val_accuracy = max(history.history['val_accuracy'])
    best_epoch = history.history['val_accuracy'].index(best_val_accuracy)
    print(f"Saved at Epoch: {best_epoch + 1}")

    val_loss = history.history['val_loss'][best_epoch]
    val_accuracy = history.history['val_accuracy'][best_epoch]
    test_loss, test_accuracy = model.evaluate(test_generator, verbose=0)

    recall = get_recall_score(model, test_generator)
    f1 = get_f1_score(model, test_generator)

    print(f"Validation Loss: {round(val_loss, 2)}")
    print(f"Validation Accuracy: {round(val_accuracy * 100, 2)}%")
    print(f"Test Loss: {round(test_loss, 2)}")
    print(f"Test Accuracy: {round(test_accuracy * 100, 2)}%")
    print(f"Macro Recall: {round(recall * 100, 2)}%")
    print(f"Macro F1: {round(f1 * 100, 2)}%")


def load_and_preprocess_image(image_path, target_size=(224, 224)):
    img = Image.open(image_path).convert('RGB')
    img = img.resize(target_size)
    img_array = np.array(img) / 255.0  # Normalize the image
    img_array = np.expand_dims(img_array, axis=0)
    return img_array


def get_predicted_label(model, path):
    img_array = load_and_preprocess_image(path)
    predictions = model.predict(img_array, verbose=0)
    predicted_class = np.argmax(predictions, axis=1)[0]
    class_labels = list(test_generator.class_indices.keys())
    return class_labels[predicted_class]


def display_prediction(model, path):
    predicted_label = get_predicted_label(model, path)
    img = Image.open(path)
    plt.imshow(img)
    plt.axis('off')
    plt.title(f"Prediction: {predicted_label}")
    plt.show()
    plt.close()


### Generate Data without augmentation

In [ ]:
y_col = 'card type' # 'labels' for all cards & 'card type' for suits
train_generator = data_generator(data[data['data set'] == 'train'], y_col=y_col, use_augmentation=False)
valid_generator = data_generator(data[data['data set'] == 'valid'], y_col=y_col, use_augmentation=False) # use_augmentation is always false
test_generator = data_generator(data[data['data set'] == 'test'], y_col=y_col, use_augmentation=False) # use_augmentation is always false

In [ ]:
test_generator.class_indices

In [ ]:
plot_images(train_generator, num_images=8)

### Custom Model v1

In [ ]:
model_shape = 'shape' if tf.__version__ >= '2.10.1' else 'input_shape'
print("TensorFlow version:", tf.__version__)

input_args = {
    model_shape: (224, 224, 3)
}

In [ ]:
model = create_custom_model_v1(**input_args, num_classes=len(train_generator.class_indices))
model = prepare_model(model, optimizer='adam', loss='categorical_crossentropy', learning_rate=0.0003, metrics=['accuracy'])

history = train_model(model, train_generator, valid_generator, epochs=20, batch_size=32, use_early_stopping = True, save_name=model.name, verbose=1)

In [ ]:
get_classification_report(model, test_generator)

In [ ]:
plot_training_history(history)

In [ ]:
plot_confusion_matrix(model, test_generator)

In [ ]:
print('Model Performance Evaluation:')
print_model_performance(model, history, valid_generator, test_generator)

In [ ]:
image_path = 'uploads/demo.jpg'
print('Displaying prediction for a sample image...')
display_prediction(model, image_path)

### Custom Model v2

In [ ]:
model = create_custom_model_v2(**input_args, num_classes=len(train_generator.class_indices))
model = prepare_model(model, optimizer='adam', loss='categorical_crossentropy', learning_rate=0.0003, metrics=['accuracy'])

history = train_model(model, train_generator, valid_generator, epochs=20, batch_size=32, use_early_stopping = True, save_name=model.name, verbose=1)

In [ ]:
get_classification_report(model, test_generator)

In [ ]:
plot_training_history(history)
plot_confusion_matrix(model, test_generator)

In [ ]:
print('Model Performance Evaluation:')
print_model_performance(model, history, valid_generator, test_generator)

In [ ]:
image_path = 'uploads/demo.jpg'
print('Displaying prediction for a sample image...')
display_prediction(model, image_path)

---

### Custom Model v3

In [ ]:
model = create_custom_model_v3(**input_args, num_classes=len(train_generator.class_indices))
model = prepare_model(model, optimizer='adam', loss='categorical_crossentropy', learning_rate=0.001,
                      metrics=['accuracy'])

history = train_model(model, train_generator, valid_generator, epochs=30, batch_size=32, use_early_stopping=True,
                      save_name=model.name, verbose=1)

In [ ]:
get_classification_report(model, test_generator)

In [ ]:
plot_training_history(history)
plot_confusion_matrix(model, test_generator)

In [ ]:
print('Model Performance Evaluation:')
print_model_performance(model, history, valid_generator, test_generator)

In [ ]:
image_path = 'uploads/demo.jpg'
print('Displaying prediction for a sample image...')
display_prediction(model, image_path)

### VGG 19

In [ ]:
vgg19_model = create_pretrained_model_vgg19(shape=(224, 224, 3), num_classes=len(train_generator.class_indices))

vgg19_model = prepare_model(vgg19_model, optimizer='adam', loss='categorical_crossentropy', learning_rate=0.0003, metrics=['accuracy'])
history = train_model(vgg19_model, train_generator, valid_generator, epochs=20, batch_size=32, use_early_stopping=True, save_name=vgg19_model.name, verbose=1)

In [ ]:
get_classification_report(vgg19_model, test_generator)

In [ ]:
plot_training_history(history)
plot_confusion_matrix(vgg19_model, test_generator)

In [ ]:
print('Model Performance Evaluation:')
print_model_performance(vgg19_model, history, valid_generator, test_generator)

In [ ]:
image_path = 'uploads/demo.jpg'
print('Displaying prediction for a sample image...')
display_prediction(vgg19_model, image_path)

### ResNet50

In [ ]:
resnet50_model = create_pretrained_model_resnet50(shape=(224, 224, 3), num_classes=len(train_generator.class_indices))
resnet50_model = prepare_model(resnet50_model, optimizer='adam', loss='categorical_crossentropy', learning_rate=0.0003, metrics=['accuracy'])
history = train_model(resnet50_model, train_generator, valid_generator, epochs=30, batch_size=16, use_early_stopping=True, save_name=resnet50_model.name, verbose=1)

In [ ]:
get_classification_report(resnet50_model, test_generator)

In [ ]:
plot_training_history(history)
plot_confusion_matrix(resnet50_model, test_generator)

In [ ]:
print('Model Performance Evaluation:')
print_model_performance(resnet50_model, history, valid_generator, test_generator)

In [ ]:
image_path = 'uploads/demo.jpg'
print('Displaying prediction for a sample image...')
display_prediction(resnet50_model, image_path)

### Generate Data with augmentation

#### !!! Augmentation is only applied to training data !!!

In [ ]:
train_generator = data_generator(data[data['data set'] == 'train'], y_col=y_col, use_augmentation=True)
valid_generator = data_generator(data[data['data set'] == 'valid'], y_col=y_col, use_augmentation=False) # here use_augmentation is always false
test_generator = data_generator(data[data['data set'] == 'test'], y_col=y_col, use_augmentation=False) # here use_augmentation is always false

data['card type'] = data['card type'].replace('xxx', 'joker')

### Custom Model v3 with Augmentation

In [ ]:
model = create_custom_model_v3(**input_args, num_classes=len(train_generator.class_indices))
model = prepare_model(model, optimizer='adam', loss='categorical_crossentropy', learning_rate=0.001,
                      metrics=['accuracy'])

history = train_model(model, train_generator, valid_generator, epochs=30, batch_size=128, use_early_stopping=True,
                      save_name="custom_model_v3_augmented", verbose=1)

In [ ]:
get_classification_report(model, test_generator)

In [ ]:
plot_training_history(history)
plot_confusion_matrix(model, test_generator)

In [ ]:
print('Model Performance Evaluation:')
print_model_performance(model, history, valid_generator, test_generator)

In [ ]:
image_path = 'uploads/demo.jpg'
print('Displaying prediction for a sample image...')
display_prediction(model, image_path)

### VGG 19 with Augmentation

In [ ]:
vgg19_model = create_pretrained_model_vgg19(shape=(224, 224, 3), num_classes=len(train_generator.class_indices))

vgg19_model = prepare_model(vgg19_model, optimizer='adam', loss='categorical_crossentropy', learning_rate=0.0003,
                            metrics=['accuracy'])
history = train_model(vgg19_model, train_generator, valid_generator, epochs=30, batch_size=32, use_early_stopping=True,
                      save_name="vgg19_pretrained_model_augmented", verbose=1)

In [ ]:
get_classification_report(vgg19_model, test_generator)

In [ ]:
plot_training_history(history)
plot_confusion_matrix(vgg19_model, test_generator)

In [ ]:
print('Model Performance Evaluation:')
print_model_performance(vgg19_model, history, valid_generator, test_generator)

In [ ]:
image_path = 'uploads/demo.jpg'
print('Displaying prediction for a sample image...')
display_prediction(vgg19_model, image_path)

### ResNet50 with Augmentation

In [ ]:
resnet50_model = create_pretrained_model_resnet50(shape=(224, 224, 3), num_classes=len(train_generator.class_indices))

resnet50_model = prepare_model(resnet50_model, optimizer='adam', loss='categorical_crossentropy', learning_rate=0.0003,
                            metrics=['accuracy'])
history = train_model(resnet50_model, train_generator, valid_generator, epochs=20, batch_size=32, use_early_stopping=True,
                      save_name="resnet50_pretrained_model_augmented", verbose=1)

In [ ]:
get_classification_report(resnet50_model, test_generator)

In [ ]:
plot_training_history(history)
plot_confusion_matrix(resnet50_model, test_generator)

In [ ]:
print('Model Performance Evaluation:')
print_model_performance(resnet50_model, history, valid_generator, test_generator)

In [ ]:
image_path = 'uploads/demo.jpg'
print('Displaying prediction for a sample image...')
display_prediction(resnet50_model, image_path)